# Municípios do Brasil

Vamos transformar a tabela de municípios do Brasil disponibilizada pelo **kelvins** no GitHub para usar ao longo desse projeto. 

In [1]:
import pandas as pd
import unicodedata
from sqlalchemy import create_engine

In [2]:
engine = create_engine('sqlite:///../database/nf-goiana.db')

In [3]:
url_municipios = 'https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/main/csv/municipios.csv'
url_ufs = 'https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/main/csv/estados.csv'

In [4]:
municipios = pd.read_csv(url_municipios)
municipios.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52,1050,62,America/Sao_Paulo
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31,4001,34,America/Sao_Paulo
2,5200100,Abadiânia,-16.19700,-48.7057,0,52,9201,62,America/Sao_Paulo
3,3100203,Abaeté,-19.15510,-45.4444,0,31,4003,37,America/Sao_Paulo
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15,401,91,America/Sao_Paulo


In [5]:
ufs = pd.read_csv(url_ufs)
ufs.head()

,codigo_uf,uf,nome,latitude,longitude,regiao
0,11,RO,Rondônia,-10.83,-63.34,Norte
1,12,AC,Acre,-8.77,-70.55,Norte
2,13,AM,Amazonas,-3.47,-65.10,Norte
3,14,RR,Roraima,1.99,-61.33,Norte
4,15,PA,Pará,-3.79,-52.48,Norte


In [6]:
ufs.drop(columns=['latitude', 'longitude', 'regiao'], inplace=True)
municipios.drop(columns=['siafi_id', 'ddd', 'fuso_horario'], inplace=True)

Como existe muitas cidades com o mesmo nome, vou criar uma lista de prioridade seguindo a regra: `capital > outros municípios` e `municípios de goiás > demais`, pois imagina-se que é mais provável que uma pessoa seja de um município que é capital do que de outros municípios, pois são mais populosos e a probabilidade dessas pessoas serem do estado de Goiás é maior pois é onde o programa é ativo (as compras precisam ser feitas no estado para concorrerem).

In [7]:
municipios = municipios.merge(ufs, on='codigo_uf')
municipios.rename(columns={'nome_x': 'municipio', 'nome_y': 'uf_nome'}, inplace=True)
municipios.head()

,codigo_ibge,municipio,latitude,longitude,capital,codigo_uf,uf,uf_nome
0,5200050,Abadia de Goiás,-16.7573,-49.4412,0,52,GO,Goiás
1,5200100,Abadiânia,-16.1970,-48.7057,0,52,GO,Goiás
2,5200134,Acreúna,-17.3960,-50.3749,0,52,GO,Goiás
3,5200159,Adelândia,-16.4127,-50.1657,0,52,GO,Goiás
4,5200175,Água Fria de Goiás,-14.9778,-47.7823,0,52,GO,Goiás


In [8]:
municipios['goias'] = [int(x == 'GO') for x in municipios.uf]
municipios.sort_values(by=['capital', 'goias'], ascending=False, inplace=True)
municipios.head()

,codigo_ibge,municipio,latitude,longitude,capital,codigo_uf,uf,uf_nome,goias
94,5208707,Goiânia,-16.68640,-49.2643,1,52,GO,Goiás,1
311,3106200,Belo Horizonte,-19.91020,-43.9266,1,31,MG,Minas Gerais,0
1117,1501402,Belém,-1.45540,-48.4898,1,15,PA,Pará,0
1301,2304400,Fortaleza,-3.71664,-38.5423,1,23,CE,Ceará,0
1762,2927408,Salvador,-12.97180,-38.5011,1,29,BA,Bahia,0


In [9]:
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

In [10]:
municipios['nome_simples'] = [strip_accents(i) for i in municipios.municipio.str.upper()]
municipios['uf_simples'] = [strip_accents(i) for i in municipios.uf_nome.str.upper()]
municipios.head()

,codigo_ibge,municipio,latitude,longitude,capital,codigo_uf,uf,uf_nome,goias,nome_simples,uf_simples
94,5208707,Goiânia,-16.68640,-49.2643,1,52,GO,Goiás,1,GOIANIA,GOIAS
311,3106200,Belo Horizonte,-19.91020,-43.9266,1,31,MG,Minas Gerais,0,BELO HORIZONTE,MINAS GERAIS
1117,1501402,Belém,-1.45540,-48.4898,1,15,PA,Pará,0,BELEM,PARA
1301,2304400,Fortaleza,-3.71664,-38.5423,1,23,CE,Ceará,0,FORTALEZA,CEARA
1762,2927408,Salvador,-12.97180,-38.5011,1,29,BA,Bahia,0,SALVADOR,BAHIA


In [11]:
municipios.drop(columns=['capital','goias'], inplace=True)
municipios.head()

,codigo_ibge,municipio,latitude,longitude,codigo_uf,uf,uf_nome,nome_simples,uf_simples
94,5208707,Goiânia,-16.68640,-49.2643,52,GO,Goiás,GOIANIA,GOIAS
311,3106200,Belo Horizonte,-19.91020,-43.9266,31,MG,Minas Gerais,BELO HORIZONTE,MINAS GERAIS
1117,1501402,Belém,-1.45540,-48.4898,15,PA,Pará,BELEM,PARA
1301,2304400,Fortaleza,-3.71664,-38.5423,23,CE,Ceará,FORTALEZA,CEARA
1762,2927408,Salvador,-12.97180,-38.5011,29,BA,Bahia,SALVADOR,BAHIA


In [12]:
municipios.to_sql(name='municipios', con=engine, index=False, if_exists='replace')

5570